# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 18 2022 2:23PM,248890,10,MSP214984,"Methapharm, Inc.",Released
1,Oct 18 2022 2:23PM,248890,10,MSP214986,"Methapharm, Inc.",Released
2,Oct 18 2022 2:23PM,248890,10,MSP214987,"Methapharm, Inc.",Released
3,Oct 18 2022 2:23PM,248890,10,MSP214988,"Methapharm, Inc.",Released
4,Oct 18 2022 2:23PM,248890,10,MSP214990,"Methapharm, Inc.",Released
5,Oct 18 2022 2:23PM,248890,10,MSP214991,"Methapharm, Inc.",Released
6,Oct 18 2022 2:23PM,248890,10,MSP214998,"Methapharm, Inc.",Released
7,Oct 18 2022 2:23PM,248890,10,MSP214999,"Methapharm, Inc.",Released
8,Oct 18 2022 2:23PM,248890,10,MSP215000,"Methapharm, Inc.",Released
9,Oct 18 2022 2:23PM,248890,10,MSP215001,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
28,248884,Released,1
29,248886,Released,16
30,248887,Released,1
31,248889,Released,6
32,248890,Released,15


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
248884,NaN,NaN,1.0
248886,NaN,NaN,16.0
248887,NaN,NaN,1.0
248889,NaN,NaN,6.0
248890,NaN,NaN,15.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
248813,0.0,0.0,1.0
248836,13.0,27.0,5.0
248837,0.0,0.0,37.0
248845,0.0,0.0,3.0
248846,0.0,0.0,18.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
248813,0,0,1
248836,13,27,5
248837,0,0,37
248845,0,0,3
248846,0,0,18


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,248813,0,0,1
1,248836,13,27,5
2,248837,0,0,37
3,248845,0,0,3
4,248846,0,0,18


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,248813,,,1
1,248836,13,27,5
2,248837,,,37
3,248845,,,3
4,248846,,,18


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 18 2022 2:23PM,248890,10,"Methapharm, Inc."
15,Oct 18 2022 2:19PM,248889,10,"Methapharm, Inc."
18,Oct 18 2022 2:19PM,248889,10,Methapharm-G
21,Oct 18 2022 2:14PM,248886,10,ISDIN Corporation
37,Oct 18 2022 2:09PM,248887,10,ISDIN Corporation
38,Oct 18 2022 2:02PM,248883,10,ISDIN Corporation
52,Oct 18 2022 2:00PM,248884,10,"Citieffe, Inc."
53,Oct 18 2022 1:32PM,248880,10,"Citieffe, Inc."
54,Oct 18 2022 1:20PM,248879,10,Ivaoes Animal Health
55,Oct 18 2022 1:10PM,248878,10,"Methapharm, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Oct 18 2022 2:23PM,248890,10,"Methapharm, Inc.",,,15
1,Oct 18 2022 2:19PM,248889,10,"Methapharm, Inc.",,,6
2,Oct 18 2022 2:19PM,248889,10,Methapharm-G,,,6
3,Oct 18 2022 2:14PM,248886,10,ISDIN Corporation,,,16
4,Oct 18 2022 2:09PM,248887,10,ISDIN Corporation,,,1
5,Oct 18 2022 2:02PM,248883,10,ISDIN Corporation,,,14
6,Oct 18 2022 2:00PM,248884,10,"Citieffe, Inc.",,,1
7,Oct 18 2022 1:32PM,248880,10,"Citieffe, Inc.",,,1
8,Oct 18 2022 1:20PM,248879,10,Ivaoes Animal Health,,,1
9,Oct 18 2022 1:10PM,248878,10,"Methapharm, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 18 2022 2:23PM,248890,10,"Methapharm, Inc.",15,,
1,Oct 18 2022 2:19PM,248889,10,"Methapharm, Inc.",6,,
2,Oct 18 2022 2:19PM,248889,10,Methapharm-G,6,,
3,Oct 18 2022 2:14PM,248886,10,ISDIN Corporation,16,,
4,Oct 18 2022 2:09PM,248887,10,ISDIN Corporation,1,,
5,Oct 18 2022 2:02PM,248883,10,ISDIN Corporation,14,,
6,Oct 18 2022 2:00PM,248884,10,"Citieffe, Inc.",1,,
7,Oct 18 2022 1:32PM,248880,10,"Citieffe, Inc.",1,,
8,Oct 18 2022 1:20PM,248879,10,Ivaoes Animal Health,1,,
9,Oct 18 2022 1:10PM,248878,10,"Methapharm, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 18 2022 2:23PM,248890,10,"Methapharm, Inc.",15,,
1,Oct 18 2022 2:19PM,248889,10,"Methapharm, Inc.",6,,
2,Oct 18 2022 2:19PM,248889,10,Methapharm-G,6,,
3,Oct 18 2022 2:14PM,248886,10,ISDIN Corporation,16,,
4,Oct 18 2022 2:09PM,248887,10,ISDIN Corporation,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 18 2022 2:23PM,248890,10,"Methapharm, Inc.",15,NaN,NaN
1,Oct 18 2022 2:19PM,248889,10,"Methapharm, Inc.",6,NaN,NaN
2,Oct 18 2022 2:19PM,248889,10,Methapharm-G,6,NaN,NaN
3,Oct 18 2022 2:14PM,248886,10,ISDIN Corporation,16,NaN,NaN
4,Oct 18 2022 2:09PM,248887,10,ISDIN Corporation,1,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 18 2022 2:23PM,248890,10,"Methapharm, Inc.",15,0.0,0.0
1,Oct 18 2022 2:19PM,248889,10,"Methapharm, Inc.",6,0.0,0.0
2,Oct 18 2022 2:19PM,248889,10,Methapharm-G,6,0.0,0.0
3,Oct 18 2022 2:14PM,248886,10,ISDIN Corporation,16,0.0,0.0
4,Oct 18 2022 2:09PM,248887,10,ISDIN Corporation,1,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4230880,73,0.0,0.0
15,995364,63,27.0,13.0
19,1493165,18,16.0,0.0
22,746604,3,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4230880,73,0.0,0.0
1,15,995364,63,27.0,13.0
2,19,1493165,18,16.0,0.0
3,22,746604,3,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,73,0.0,0.0
1,15,63,27.0,13.0
2,19,18,16.0,0.0
3,22,3,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,73.0
1,15,Released,63.0
2,19,Released,18.0
3,22,Released,3.0
4,10,Executing,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,19,22
Status,,,,
Completed,0.0,13.0,0.0,0.0
Executing,0.0,27.0,16.0,0.0
Released,73.0,63.0,18.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,19,22
0,Completed,0.0,13.0,0.0,0.0
1,Executing,0.0,27.0,16.0,0.0
2,Released,73.0,63.0,18.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,19,22
0,Completed,0.0,13.0,0.0,0.0
1,Executing,0.0,27.0,16.0,0.0
2,Released,73.0,63.0,18.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()